In [3]:
#Importing libraries
import pandas as pd
import numpy as np
from scipy.optimize import fmin_slsqp
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import random

#Listing the existing files in the dir
for root, dirs, files in os.walk("."):  
    for filename in files:
        print(filename)

lorenz.py
markdown_python.md
jupyterlab.md
big.csv
Lorenz.ipynb
jupyterlab-slides.pdf
code.ipynb
Untitled.ipynb
example.csv
zika_assembled_genomes.fasta
1024px-Hubble_Interacting_Galaxy_AM_0500-620_(2008-04-24).jpg
Museums_in_DC.geojson
Dockerfile
README.md
bar.vl.json
japan_meterological_agency_201707211555.json
iris.csv
Data.ipynb
lorenz.py
Cpp.ipynb
R.ipynb
bqplot.ipynb
Fasta.ipynb
Lorenz.ipynb
pandas.ipynb
marie.png
xtensor.png
xeus-cling.png
xwidgets.png
audio.wav
hpv_summary_3_20_13_distribute.csv
p53_calls_pancancer.maf
amino_acids.csv
nationwidechildrens.org_auxiliary_hnsc.txt
gene_coords.csv
HNSCC_molecular_validation.csf
c2.cp.v3.0.symbols_edit.csv
diseaseStudy.txt
p53_calls_pancancer.csv
nationwidechildrens.org_clinical_patient_hnsc.txt
HNSCC_molecular_validation.maf
HNSCC_molecular_validation_matrix.csv
hgnc_regions.txt
HGNC_chr_pos.txt
HGNC_Genes
pitt_broad_data_update.csv
maf.csv
meta.csv
cn_regions.csv
TCGA_followup_UPMC.csv
pitt_broad_data.csv
all_thresholded.by_genes.t

In [4]:
df2 = pd.read_csv('./example.csv')
inputss = pd.DataFrame(df2.iloc[:,1:3])
outputss = pd.DataFrame(df2.iloc[:,3])

In [14]:
input_w = np.zeros((inputss.shape[1], 1), dtype=np.float)  # input weights

denominator = np.dot(inputss, input_w)
denominator

array([[0.],
       [0.],
       [0.]])

In [ ]:
class DEA(object):

    def __init__(self, inputs, outputs):
        """
        Initialize the DEA object with input data
        n = number of entities (observations)
        m = number of inputs (variables, features)
        r = number of outputs
        :param inputs: inputs, n x m numpy array
        :param outputs: outputs, n x r numpy array
        :return: self
        """

        # supplied data
        self.inputs = inputs
        self.outputs = outputs

        # parameters
        self.n = inputs.shape[0]
        self.m = inputs.shape[1]
        self.r = outputs.shape[1]

        # iterators
        self.unit_ = range(self.n)
        self.input_ = range(self.m)
        self.output_ = range(self.r)

        # result arrays
        self.output_w = np.zeros((self.r, 1), dtype=np.float)  # output weights
        self.input_w = np.zeros((self.m, 1), dtype=np.float)  # input weights
        self.lambdas = np.zeros((self.n, 1), dtype=np.float)  # unit efficiencies
        self.efficiency = np.zeros_like(self.lambdas)  # thetas

        # names
        self.names = []

    def __efficiency(self, unit):
        """
        Efficiency function with already computed weights
        :param unit: which unit to compute for
        :return: efficiency
        """

        # compute efficiency
        denominator = np.dot(self.inputs, self.input_w)
        numerator = np.dot(self.outputs, self.output_w)

        return (numerator/denominator)[unit]

    def __target(self, x, unit):
        """
        Theta target function for one unit
        :param x: combined weights
        :param unit: which production unit to compute
        :return: theta
        """
        in_w, out_w, lambdas = x[:self.m], x[self.m:(self.m+self.r)], x[(self.m+self.r):]  # unroll the weights
        denominator = np.dot(self.inputs[unit], in_w)
        numerator = np.dot(self.outputs[unit], out_w)

        return numerator/denominator

    def __constraints(self, x, unit):
        """
        Constraints for optimization for one unit
        :param x: combined weights
        :param unit: which production unit to compute
        :return: array of constraints
        """

        in_w, out_w, lambdas = x[:self.m], x[self.m:(self.m+self.r)], x[(self.m+self.r):]  # unroll the weights
        constr = []  # init the constraint array

        # for each input, lambdas with inputs
        for input in self.input_:
            t = self.__target(x, unit)
            lhs = np.dot(self.inputs[:, input], lambdas)
            cons = t*self.inputs[unit, input] - lhs
            constr.append(cons)

        # for each output, lambdas with outputs
        for output in self.output_:
            lhs = np.dot(self.outputs[:, output], lambdas)
            cons = lhs - self.outputs[unit, output]
            constr.append(cons)

        # for each unit
        for u in self.unit_:
            constr.append(lambdas[u])

        return np.array(constr)

    def __optimize(self):
        """
        Optimization of the DEA model
        Use: http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.optimize.linprog.html
        A = coefficients in the constraints
        b = rhs of constraints
        c = coefficients of the target function
        :return:
        """
        d0 = self.m + self.r + self.n
        # iterate over units
        for unit in self.unit_:
            # weights
            x0 = np.random.rand(d0) - 0.5
            x0 = fmin_slsqp(self.__target, x0, f_ieqcons=self.__constraints, args=(unit,))
            # unroll weights
            self.input_w, self.output_w, self.lambdas = x0[:self.m], x0[self.m:(self.m+self.r)], x0[(self.m+self.r):]
            self.efficiency[unit] = self.__efficiency(unit)

    def name_units(self, names):
        """
        Provide names for units for presentation purposes
        :param names: a list of names, equal in length to the number of units
        :return: nothing
        """

        assert(self.n == len(names))

        self.names = names

    def fit(self):
        """
        Optimize the dataset, generate basic table
        :return: table
        """

        self.__optimize()  # optimize
        print("---------------------------\n")       

        m = {}
        l = {}         
        for n, eff in enumerate(self.efficiency):           
            if eff >= 1.:
                m.update({self.names[n]: eff[0]}) 
            else:
                l.update({self.names[n]: eff[0]})                 
        print("Efficient units:")     
        print(str(m).replace("{","").replace("}", ""))       
        print("\n")    
        print("Inefficient units:")
        print(str(l).replace("{","").replace("}", ""))         
            

dea = DEA(inpt_arr,outpt_arr)
dea.name_units(comp)
dea.fit()